# Overview

This notebook includes prototypical code for the operator overview write-up. It does not depend on any exisiting code in DiffEqOperators, but it should be easy to modify the current codebase to achieve the same functionality.

To make things simpler, everything is assumed to use `Float64` datatype. I will also always use the out-of-place convention (i.e. `*` instead of `A_mul_B!`).

The naming of different operators will use the following convention, as in the writeup:

- `L` denotes a (quasi)linear operator.

- `A` denotes an operator that can generally (but not necessarily) be affine.

- `b` denotes the bias of an affine operator

- `M` for raw matrices (`AbstractMatrix`) and `x`, `y`, `u`, etc for vectors.

In [1]:
import Base: +, -, *, \

abstract type DiffEqOperator end
abstract type DiffEqLinearOperator <: DiffEqOperator end

update_coefficients!(::DiffEqOperator,u,p,t) = nothing; # Default update behavior

# 1. Abstract operator interface

Below is a simplified operator interface used in my new interface draft. Basically we want to express lazy addition and multiplication of linear operators naturally using `+` and `*`. The `as_array` interface returns the most suitable (dense/sparse) representation of the underlying operator as an `AbstractMatrix` (or should we treat this as a type conversion and just use `AbstractMatrix`?).

The affine operator is defined in such a way that arithmetic on them and linear operators always yield an `DiffEqAffineOperator`.

In [2]:
# We should improve type stability by including more inferrable fields in the type signature
# But for the sake of this notebook I'll make things simple
struct DiffEqArrayOperator <: DiffEqLinearOperator
    M::AbstractMatrix{Float64}
end

*(L::DiffEqArrayOperator, x::Vector{Float64}) = L.M * x
as_array(L::DiffEqArrayOperator) = L.M

struct DiffEqOperatorCombination <: DiffEqLinearOperator
    cs::Tuple{Vararg{Float64}} # Coefficients
    Ls::Tuple{Vararg{DiffEqLinearOperator}}
end

update_coefficients!(L::DiffEqOperatorCombination,u,p,t) = foreach(Lk -> update_coefficients!(Lk,u,p,t), L.Ls)
*(L::DiffEqOperatorCombination, x::Vector{Float64}) = sum(ck * (Lk*x) for (ck,Lk) in zip(L.cs,L.Ls))
as_array(L::DiffEqOperatorCombination) = sum(ck * as_array(Lk) for (ck,Lk) in zip(L.cs,L.Ls))

struct DiffEqOperatorComposition <: DiffEqLinearOperator
    Ls::Tuple{Vararg{DiffEqLinearOperator}}
end

update_coefficients!(L::DiffEqOperatorComposition,u,p,t) = foreach(Lk -> update_coefficients!(Lk,u,p,t), L.Ls)
*(L::DiffEqOperatorComposition, x::Vector{Float64}) = foldl((u, Lk) -> Lk*u, x, L.Ls)
as_array(L::DiffEqOperatorComposition) = prod(as_array, reverse(L.Ls))

struct DiffEqAffineOperator <: DiffEqOperator
    L::DiffEqLinearOperator
    b::Vector{Float64}
end

# TODO: time-dependent bias
update_coefficients!(A::DiffEqAffineOperator,u,p,t) = update_coefficients!(A.L,u,p,t)
*(A::DiffEqAffineOperator, x::Vector{Float64}) = A.L * x + A.b;

In [3]:
# Operator arithmetic
# Addition
+(L1::DiffEqOperatorCombination, L2::DiffEqOperatorCombination) = DiffEqOperatorCombination(
    (L1.cs...,L2.cs...), (L1.Ls...,L2.Ls...))
+(L1::DiffEqOperatorCombination, L2::DiffEqLinearOperator) = DiffEqOperatorCombination((L1.cs...,1.0), (L1.Ls...,L2))
+(L1::DiffEqLinearOperator, L2::DiffEqOperatorCombination) = L2 + L1
+(L1::DiffEqLinearOperator, L2::DiffEqLinearOperator) = DiffEqOperatorCombination((1.0,1.0), (L1,L2))

+(A1::DiffEqAffineOperator, L2::DiffEqLinearOperator) = DiffEqAffineOperator(A1.L + L2, A1.b)
+(L1::DiffEqLinearOperator, A2::DiffEqAffineOperator) = A2 + L1
+(A1::DiffEqAffineOperator, A2::DiffEqAffineOperator) = DiffEqAffineOperator(A1.L + A2.L, A1.b + A2.b)

# Scalar multiplication
*(α::Float64, L::DiffEqOperatorCombination) = DiffEqOperatorCombination(α.*L.cs, L.Ls)
*(α::Float64, L::DiffEqLinearOperator) = DiffEqOperatorCombination((α,), (L,))
*(α::Float64, A::DiffEqAffineOperator) = DiffEqAffineOperator(α * A.L, α * A.b)
*(A::DiffEqOperator, α::Float64) = α * A

# Subtraction/unary minus
-(A::DiffEqOperator) = (-1.0) * A
-(A1::DiffEqOperator, A2::DiffEqOperator) = A1 + (-A2)

# Multiplication
# Note the application order
*(L1::DiffEqOperatorComposition, L2::DiffEqOperatorComposition) = DiffEqOperatorComposition((L2.Ls..., L1.Ls...))
*(L1::DiffEqLinearOperator, L2::DiffEqOperatorComposition) = DiffEqOperatorComposition((L2.Ls..., L1))
*(L1::DiffEqOperatorComposition, L2::DiffEqLinearOperator) = DiffEqOperatorComposition((L2, L1.Ls...))
*(L1::DiffEqLinearOperator, L2::DiffEqLinearOperator) = DiffEqOperatorComposition((L2, L1))

*(L1::DiffEqLinearOperator, A2::DiffEqAffineOperator) = DiffEqAffineOperator(L1 * A2.L, L1 * A2.b)
*(A1::DiffEqAffineOperator, L2::DiffEqLinearOperator) = DiffEqAffineOperator(A1.L * L2, A1.b)
*(A1::DiffEqAffineOperator, A2::DiffEqAffineOperator) = DiffEqAffineOperator(A1.L * A2.L, A1.L * A2.b + A1.b)

# Right division (i.e. linear solve)
# In the full version we should also include interface to lazy solvers
\(L::DiffEqLinearOperator, y::Vector{Float64}) = as_array(L) \ y
\(A::DiffEqAffineOperator, y::Vector{Float64}) = A.L \ (y - A.b);

# 2. Scalar field ("coefficients") operator

Represents the linear operator

$$ \alpha: u(x) \rightarrow \alpha(x)u(x) $$

$\alpha$ may in general also depend on `(u,p,t)`. In the constructor for `ScalarField`, `α` has the signature `α(u,p,t,x) -> y`. The grid type is just `Vector{Float64}`; we can implement a sophisticated grid type after the discussions [here](https://github.com/JuliaDiffEq/PDERoadmap/issues/4).

Note that in the constructor `αgrid` is uninitialized. It only obtains value after `update_coefficients!` is called. This should not be a real problem though as the `L(u,p,t)` interface always calls `update_coefficients!` on itself.

In [4]:
struct ScalarField <: DiffEqLinearOperator
    α
    grid::Vector{Float64}
    αgrid::Vector{Float64}
end
ScalarField(α, grid::Vector{Float64}) = ScalarField(α, grid, similar(grid)) # αgrid is uninitialized
function update_coefficients!(L::ScalarField,u,p,t)
    for i in 1:length(grid)
        L.αgrid[i] = L.α(u,p,t,grid[i])
    end
end

*(L::ScalarField, u::Vector{Float64}) = L.αgrid .* u
as_array(L::ScalarField) = Diagonal(L.αgrid);

Example:

$$\alpha(x) = Kx^2,\quad K\in\mathbb{R}$$

In [5]:
grid = collect(0.0:1.0:5.0)
α = (u,p,t,x) -> p * x^2
Lα = ScalarField(α, grid)

ScalarField(#11, [0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [2.32555e-315, 7.15942e-316, 2.32555e-315, 7.16065e-316, 7.1655e-316, 0.0])

In [6]:
u = sin.(grid)
p = 1.0
t = 0.0

update_coefficients!(Lα,u,p,t)
@show Lα.αgrid
@show Lα * u;

Lα.αgrid = [0.0, 1.0, 4.0, 9.0, 16.0, 25.0]
Lα * u = [0.0, 0.841471, 3.63719, 1.27008, -12.1088, -23.9731]


In [7]:
p = 2.0
update_coefficients!(Lα,u,p,t)
@show Lα.αgrid
@show Lα * u;

Lα.αgrid = [0.0, 2.0, 8.0, 18.0, 32.0, 50.0]
Lα * u = [0.0, 1.68294, 7.27438, 2.54016, -24.2177, -47.9462]


# 3. Discretization of differenctial operators

The 2nd-order central difference approximation to $\partial_x^2$ is hand-coded here. The general case can be modified from `DerivativeOperator` from DiffEqOperators.

The 1st-order upwind difference approximation to $\partial_x$ is also included here but operations on `FirstDifferenceUpwind` will throw an error (because the "bare" operator does not know which is the upwind direction). It is meant as a placeholder and should be used in conjunction with prepended scalar field/coefficients.

In this notebook I shall assume the grid to be uniform and represents the interior points. The grid spacing is calculated as `dx = grid[2] - grid[1]`. While this is a bit ugly, the interface will be much better once we incorporate our own `Interval` or `Grid` types.

In [8]:
struct SecondDifferenceCentral <: DiffEqLinearOperator
    grid::Vector{Float64}
end

function *(L::SecondDifferenceCentral, x::Vector{Float64})
    m = length(L.grid)
    dx = L.grid[2] - L.grid[1]
    [x[i] + x[i+2] - 2*x[i+1] for i in 1:m] / dx^2
end
function as_array(L::SecondDifferenceCentral)
    m = length(L.grid)
    dx = L.grid[2] - L.grid[1]
    spdiagm((ones(m), -2*ones(m), ones(m)), (0,1,2)) / dx^2
end

struct FirstDifferenceUpwind <: DiffEqLinearOperator
    grid::Vector{Float64}
end

struct UpwindDirectionUnknownError end
*(::FirstDifferenceUpwind, x) = throw(UpwindDirectionUnknownError())
as_array(::FirstDifferenceUpwind) = throw(UpwindDirectionUnknownError());

We're now in a position to implement the function composition interface

```julia
α1 = (u,p,t,x) -> ...
α2 = (u,p,t,x) -> ...
L = α1 * FirstDifferenceUpwind(grid) + α2 * SecondDifferenceCentral(grid)
```

For the central differencing operators, we can simply interpret `α2 * SecondDifferenceCentral(grid)` as the composition of `ScalarField(α2,grid)` and `SecondDifferenceCentral(grid)`. On the other hand, for the upwind operators we need to fuse them into a composite type because the scalar field/coefficient determines the differencing directions.

Question: what if we want an α that is a functor?

In [9]:
# Central differencing
*(α::Function, L::SecondDifferenceCentral) = ScalarField(α, L.grid) * L

# Upwind differencing
struct UpwindOperator{C,O} <: DiffEqLinearOperator
    coeffs::C
    raw_operator::O
end
*(α::Float64, L::FirstDifferenceUpwind) = UpwindOperator(α, L)
*(α::Function, L::FirstDifferenceUpwind) = UpwindOperator(ScalarField(α, L.grid), L)
update_coefficients!(L::UpwindOperator{ScalarField, O},u,p,t) where {O} = update_coefficients!(L.coeffs,u,p,t)

# In the production code, *(L::UpwindOperator, x) will be generic (uses L.stencil_up 
# and L.stencil_down, for example). For this notebook I will simply implmement the 
# specific case for FirstDifferenceUpwind
function *(L::UpwindOperator{Float64, FirstDifferenceUpwind}, x::Vector{Float64})
    m = length(L.raw_operator.grid)
    dx = L.raw_operator.grid[2] - L.raw_operator.grid[1]
    if L.coeffs > 0 # right drift
        L.coeffs * [x[i+1] - x[i] for i in 1:m] / dx
    else # left drift
        L.coeffs * [x[i+2] - x[i+1] for i in 1:m] / dx
    end
end
function *(L::UpwindOperator{ScalarField, FirstDifferenceUpwind}, x::Vector{Float64})
    m = length(L.raw_operator.grid)
    dx = L.raw_operator.grid[2] - L.raw_operator.grid[1]
    out = zeros(m)
    for i in 1:m
        c = L.coeffs.αgrid[i]
        if c > 0 # right drift
            out[i] = c/dx * (x[i+1] - x[i])
        else # left drift
            out[i] = c/dx * (x[i+2] - x[i+1])
        end
    end
    out
end

function as_array(L::UpwindOperator{Float64, FirstDifferenceUpwind})
    m = length(L.raw_operator.grid)
    dx = L.raw_operator.grid[2] - L.raw_operator.grid[1]
    if L.coeffs > 0 # right drift
        spdiagm((-ones(m), ones(m), zeros(m)), (0,1,2)) / dx
    else # left drift
        spdiagm((zeros(m), -ones(m), ones(m)), (0,1,2)) / dx
    end    
end
function as_array(L::UpwindOperator{ScalarField, FirstDifferenceUpwind})
    m = length(L.raw_operator.grid)
    dx = L.raw_operator.grid[2] - L.raw_operator.grid[1]
    out = zeros(m,m+2)
    for j = 1:m
        c = L.coeffs.αgrid[j]
        if c > 0 # right drift
            out[j,j] = -c/dx
            out[j,j+1] = c/dx
        else # left drift
            out[j,j+1] = -c/dx
            out[j,j+2] = c/dx
        end
    end
    sparse(out)
end;

# TODO: implement scalar/ScalarField multiplication on UpwindOperator

Example

In [10]:
grid = collect(0.0:1.0:5.0)
u = rand(8)
α1 = (u,p,t,x) -> 1.01*x
α2 = (u,p,t,x) -> x^2
L = α1 * FirstDifferenceUpwind(grid) + α2 * SecondDifferenceCentral(grid)

DiffEqOperatorCombination((1.0, 1.0), (UpwindOperator{ScalarField,FirstDifferenceUpwind}(ScalarField(#19, [0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [8.29573e-316, 9.88267e-316, 2.23096e-315, 9.17226e-316, 8.14163e-316, 2.17222e-315]), FirstDifferenceUpwind([0.0, 1.0, 2.0, 3.0, 4.0, 5.0])), DiffEqOperatorComposition((SecondDifferenceCentral([0.0, 1.0, 2.0, 3.0, 4.0, 5.0]), ScalarField(#21, [0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [2.23108e-315, 7.20971e-316, 7.19168e-316, 2.23108e-315, 2.23108e-315, 2.24314e-315])))))

(Again, should do something about pprint these composite operators)

In [11]:
p = nothing; t = 0.0
update_coefficients!(L,u,p,t)
L

DiffEqOperatorCombination((1.0, 1.0), (UpwindOperator{ScalarField,FirstDifferenceUpwind}(ScalarField(#19, [0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [0.0, 1.01, 2.02, 3.03, 4.04, 5.05]), FirstDifferenceUpwind([0.0, 1.0, 2.0, 3.0, 4.0, 5.0])), DiffEqOperatorComposition((SecondDifferenceCentral([0.0, 1.0, 2.0, 3.0, 4.0, 5.0]), ScalarField(#21, [0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [0.0, 1.0, 4.0, 9.0, 16.0, 25.0])))))

In [12]:
# Should check this
L * u

6-element Array{Float64,1}:
  0.0     
 -0.227745
  0.993775
  2.49862 
 -9.00639 
  8.34501 

In [14]:
full(as_array(L))

6×8 Array{Float64,2}:
 0.0   0.0    0.0    0.0     0.0     0.0     0.0    0.0
 0.0  -0.01  -0.99   1.0     0.0     0.0     0.0    0.0
 0.0   0.0    1.98  -5.98    4.0     0.0     0.0    0.0
 0.0   0.0    0.0    5.97  -14.97    9.0     0.0    0.0
 0.0   0.0    0.0    0.0    11.96  -27.96   16.0    0.0
 0.0   0.0    0.0    0.0     0.0    19.95  -44.95  25.0

# 4. Boundary extrapolation operator $Q$

Question: is it OK to shorthand "boundary extrapolation operator" as "BEOperator" or simply "BE"?

A generic $Q$ from the generic boundary condition $Bu = b$ can be a bit difficult to implement, but the simple case of Dirichlet/Neumann BC is easy to handle.

It should be easy to modify `AbsorbingBoundaryMap` and `ReflectingBoundaryMap` to incorporate boundaries that are absorbing at one end and reflecting at another.

The non-zero Dirichlet/Neumann Qs do not have their own type. Instead we construct the operator as an affine map, with `AbsorbingBoundaryMap` and `ReflectingBoundaryMap` its linear part.

In [6]:
struct AbsorbingBE <: DiffEqLinearOperator
    m::Int # number of interior points
end

*(Q::AbsorbingBE, x::Vector{Float64}) = [0.0; x; 0.0]
as_array(Q::AbsorbingBE) = sparse([zeros(Q.m)'; eye(Q.m); zeros(Q.m)'])

struct ReflectingBE <: DiffEqLinearOperator
    m::Int # number of interior points
end

*(Q::ReflectingBE, x::Vector{Float64}) = [x[1]; x; x[end]]
as_array(Q::ReflectingBE) = sparse([1.0 zeros(Q.m-1)'; eye(Q.m); zeros(Q.m-1)' 1.0]);

In [7]:
function Dirichlet_BE(m::Int, bl::Float64, br::Float64)
    # y[1] = bl, y[end] = br
    L = AbsorbingBE(m)
    b = [bl; zeros(m); br]
    DiffEqAffineOperator(L, b)
end

function Neumann_BE(m::Int, dx::Float64, bl::Float64, br::Float64)
    # (y[2] - y[1])/dx = bl, (y[end] - y[end-1])/dx = br
    L = ReflectingBE(m)
    b = [-bl*dx; zeros(m); br*dx]
    DiffEqAffineOperator(L, b)
end;

Examples of non-zero BC:

In [8]:
dx = 1.0
u = [1.,2.,3.,4.]
QD = Dirichlet_BE(4, 10.0, 20.0)
QN = Neumann_BE(4, dx, 1.0, 2.0);

In [9]:
QD * u

6-element Array{Float64,1}:
 10.0
  1.0
  2.0
  3.0
  4.0
 20.0

In [10]:
QN * u

6-element Array{Float64,1}:
 0.0
 1.0
 2.0
 3.0
 4.0
 6.0

## 5. Constructing operators for the Fokker-Planck equations

The most general case is in section 3.5 of the write-up:

$$ \mathcal{L} = \mu(x)\partial_x + \frac{\sigma(x)^2}{2}\partial_{xx} $$

where the drift term is discretized using upwind operators as described in section 2:

$$ \mu(x)\partial_x = \mu^+(x)\partial_x^+ + \mu^-(x)\partial_x^- $$

The discretized operators are expressed as the composition of an interior stencil convolution operator `L` and boundary extrapolation operator `Q` (they are generally affine). The drift and diffusion coefficients can be expressed as diagonal matrices (wrapped in a `DiffEqArrayOperator`).

The script below can be modified to represent each of the scenarios described in secition 3 of the write-up (except the mixed-BC case).

In [13]:
# # The grid
# N = 10
# dx = 1.0
# xs = collect(1:N) * dx # interior nodes

# # Discretization of the differential operators
# L1p = DriftOperator(dx, N, true)
# L1m = DriftOperator(dx, N, false)
# L2 = DiffusionOperator(dx, N)

# # Boundary operators
# Q = AbsorbingBE(N)
# # Q = Neumann_BE(N, dx, 1.0, 2.0)

# # Coefficients
# mu = rand(N)
# mup = [mu[i] > 0.0 ? mu[i] : 0.0 for i in 1:N]
# mum = [mu[i] < 0.0 ? mu[i] : 0.0 for i in 1:N]
# sigma = rand(N) + 1.0

# # Construct the final product
# Ldrift = DiffEqArrayOperator(Diagonal(mup)) * L1p + DiffEqArrayOperator(Diagonal(mum)) * L1m
# Ldiffusion = DiffEqArrayOperator(Diagonal(sigma.^2 / 2)) * L2
# A = (Ldrift + Ldiffusion) * Q

(The standard printout for the composed operator is a bit messy. Should probably implement `Base.show` for the composition types)

In [12]:
# # Solve the HJBE (rI - A)u = x
# r = 0.05
# LHS = DiffEqArrayOperator(r*speye(N)) - A
# u = LHS \ xs